# FastAPI Server Testing Notebook

This notebook tests the Titanic survival prediction FastAPI server.

## Prerequisites
- FastAPI server should be running on `http://localhost:8000`
- Start the server with: `uv run fastapi dev .\src\inference.py --`

In [21]:
# Import required libraries
import requests
import json
import pandas as pd
from pprint import pprint
import time
from dotenv import load_dotenv
import os

In [26]:
# Configuration
load_dotenv()
BASE_URL = "http://localhost:8000"
API_KEY = os.getenv("API_KEY")  # Default API key
# Headers for authentication
headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

## 1. Health Check

First, let's verify that the server is running and healthy.

In [23]:
# Test health endpoint
try:
    response = requests.get(f"{BASE_URL}/health")
    print(f"Status Code: {response.status_code}")
    print(f"Response: {response.json()}")
    
    if response.status_code == 200:
        print("✅ Server is healthy!")
    else:
        print("❌ Server health check failed!")
        
except requests.exceptions.ConnectionError:
    print("❌ Could not connect to server. Make sure it's running on http://localhost:8000")
except Exception as e:
    print(f"❌ Error: {e}")

Status Code: 200
Response: {'status': 'healthy', 'model_loaded': True, 'model_name': 'random_forest', 'timestamp': '2025-06-02T04:26:20.212738'}
✅ Server is healthy!


## 3. Single Prediction Tests

Test individual passenger predictions with different scenarios.

In [46]:
# Test cases for single predictions
test_passengers =  [{
        "PassengerId": 1,
        "Pclass": 3,
        "Name": "Braund, Mr. Owen Harris",
        "Sex": "male",
        "Age": 22.0,
        "SibSp": 1,
        "Parch": 0,
        "Ticket": "A/5 21171",
        "Fare": 7.25,
        "Cabin": None,
        "Embarked": "S",
    }]

In [47]:
# Test single predictions
print("🧪 Testing Single Predictions\n")

for test_case in test_passengers:    
    try:
        response = requests.post(
            f"{BASE_URL}/predict",
            headers=headers,
            json=test_case
        )
        
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            prediction = result['prediction']
            
            print(f"✅ Prediction: {'Survived' if prediction == 1 else 'Did not survive'}")
        else:
            print(f"❌ Failed: {response.text}")
            
    except Exception as e:
        print(f"❌ Error: {e}")
    
    print("-" * 50)

🧪 Testing Single Predictions

Status Code: 200
✅ Prediction: Did not survive
--------------------------------------------------
Status Code: 200
✅ Prediction: Did not survive
--------------------------------------------------


## 4. Batch Prediction Tests

Test multiple passengers at once using the batch prediction endpoint.

In [48]:
batch_data =  [{
        "PassengerId": 1,
        "Pclass": 3,
        "Name": "Braund, Mr. Owen Harris",
        "Sex": "male",
        "Age": 22.0,
        "SibSp": 1,
        "Parch": 0,
        "Ticket": "A/5 21171",
        "Fare": 7.25,
        "Cabin": None,
        "Embarked": "S",
    },
               {
        "PassengerId": 1,
        "Pclass": 3,
        "Name": "Braund, Mr. Owen Harris",
        "Sex": "male",
        "Age": 22.0,
        "SibSp": 1,
        "Parch": 0,
        "Ticket": "A/5 21171",
        "Fare": 7.25,
        "Cabin": None,
        "Embarked": "S",
    }
               ]

In [61]:
print("🧪 Testing Batch Predictions\n")
print(f"Sending {len(batch_data)} passengers for batch prediction...")

try:
    response = requests.post(
        f"{BASE_URL}/predict/batch",
        headers=headers,
        json={"passengers": batch_data}
    )
    
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        results = response.json()
        predictions = results['predictions']
        
        print(f"✅ Batch prediction successful!")
        print(f"   Processed {len(predictions)} passengers")
        print(predictions)
        # Display results in a table format
        print("\n📊 Batch Results:")
        for i in range(len(predictions)):
            pred = predictions[i]["prediction"]
            passenger_id = predictions[i]["passenger_id"]
            status = 'Survived' if pred == 1 else 'Did not survive'
            print(f"Passenger {passenger_id}: {status}")
    else:
        print(f"❌ Batch prediction failed: {response.text}")
        
except Exception as e:
    print(f"❌ Error: {e}")

🧪 Testing Batch Predictions

Sending 2 passengers for batch prediction...
Status Code: 200
✅ Batch prediction successful!
   Processed 2 passengers
[{'prediction': 0, 'passenger_id': 1, 'prediction_time': 0.03180193901062012}, {'prediction': 0, 'passenger_id': 1, 'prediction_time': 0.026384830474853516}]

📊 Batch Results:
Passenger 1: Did not survive
Passenger 1: Did not survive
Status Code: 200
✅ Batch prediction successful!
   Processed 2 passengers
[{'prediction': 0, 'passenger_id': 1, 'prediction_time': 0.03180193901062012}, {'prediction': 0, 'passenger_id': 1, 'prediction_time': 0.026384830474853516}]

📊 Batch Results:
Passenger 1: Did not survive
Passenger 1: Did not survive


## 5. Error Handling Tests

Test various error scenarios to ensure proper validation and error messages.

In [63]:
# Test error scenarios
error_test_cases = [
    {
        "name": "Invalid Pclass",
        "data": {"Pclass": 4, "Sex": "male", "Age": 30},
        "expected_error": "Pclass validation"
    },
    {
        "name": "Invalid Age",
        "data": {"Pclass": 1, "Sex": "female", "Age": 200},
        "expected_error": "Age validation"
    },
    {
        "name": "Invalid Sex",
        "data": {"Pclass": 2, "Sex": "unknown", "Age": 25},
        "expected_error": "Sex validation"
    },
    {
        "name": "Missing required fields",
        "data": {"Age": 30},
        "expected_error": "Missing Pclass and Sex"
    }
]

print("🧪 Testing Error Handling\n")

for test_case in error_test_cases:
    print(f"Testing: {test_case['name']}")
    print(f"Expected: {test_case['expected_error']}")
    
    try:
        response = requests.post(
            f"{BASE_URL}/predict",
            headers=headers,
            json=test_case['data']
        )
        
        if response.status_code == 422:
            print("✅ Correctly caught validation error:")
            print(f"   {test_case}")
        elif response.status_code != 200:
            print(f"✅ Correctly returned error (status {response.status_code}):")
            print(f"   {response.text}")
        else:
            print(f"⚠️  Unexpected success - should have failed")
            print(f"   Response: {response.json()}")
            
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
    
    print("-" * 40)

🧪 Testing Error Handling

Testing: Invalid Pclass
Expected: Pclass validation
✅ Correctly caught validation error:
   {'name': 'Invalid Pclass', 'data': {'Pclass': 4, 'Sex': 'male', 'Age': 30}, 'expected_error': 'Pclass validation'}
----------------------------------------
Testing: Invalid Age
Expected: Age validation
✅ Correctly caught validation error:
   {'name': 'Invalid Age', 'data': {'Pclass': 1, 'Sex': 'female', 'Age': 200}, 'expected_error': 'Age validation'}
----------------------------------------
Testing: Invalid Sex
Expected: Sex validation
✅ Correctly caught validation error:
   {'name': 'Invalid Sex', 'data': {'Pclass': 2, 'Sex': 'unknown', 'Age': 25}, 'expected_error': 'Sex validation'}
----------------------------------------
Testing: Missing required fields
Expected: Missing Pclass and Sex
✅ Correctly caught validation error:
   {'name': 'Missing required fields', 'data': {'Age': 30}, 'expected_error': 'Missing Pclass and Sex'}
----------------------------------------


## 6. Authentication Tests

Test API key authentication.

In [64]:
print("🧪 Testing Authentication\n")

# Test without API key
print("1. Testing request without API key:")
try:
    response = requests.post(
        f"{BASE_URL}/predict",
        headers={"Content-Type": "application/json"},
        json={"Pclass": 1, "Sex": "male", "Age": 30}
    )
    
    if response.status_code == 401:
        print("✅ Correctly rejected request without API key")
        print(f"   Response: {response.json()}")
    else:
        print(f"⚠️  Unexpected response: {response.status_code}")
        
except Exception as e:
    print(f"❌ Error: {e}")

print("\n2. Testing request with invalid API key:")
try:
    invalid_headers = {
        "X-API-Key": "invalid-key",
        "Content-Type": "application/json"
    }
    response = requests.post(
        f"{BASE_URL}/predict",
        headers=invalid_headers,
        json={"Pclass": 1, "Sex": "male", "Age": 30}
    )
    
    if response.status_code == 401:
        print("✅ Correctly rejected request with invalid API key")
        print(f"   Response: {response.json()}")
    else:
        print(f"⚠️  Unexpected response: {response.status_code}")
        
except Exception as e:
    print(f"❌ Error: {e}")

🧪 Testing Authentication

1. Testing request without API key:
⚠️  Unexpected response: 403

2. Testing request with invalid API key:
⚠️  Unexpected response: 403


## 7. Performance Test

Test response times and server performance.

In [66]:
print("🧪 Testing Performance\n")

# Test single prediction performance
test_data = {
        "PassengerId": 1,
        "Pclass": 3,
        "Name": "Braund, Mr. Owen Harris",
        "Sex": "male",
        "Age": 22.0,
        "SibSp": 1,
        "Parch": 0,
        "Ticket": "A/5 21171",
        "Fare": 7.25,
        "Cabin": None,
        "Embarked": "S",
    }

num_requests = 10
response_times = []

print(f"Making {num_requests} consecutive requests...")

for i in range(num_requests):
    start_time = time.time()
    
    try:
        response = requests.post(
            f"{BASE_URL}/predict",
            headers=headers,
            json=test_data
        )
        
        end_time = time.time()
        response_time = (end_time - start_time) * 1000  # Convert to milliseconds
        
        if response.status_code == 200:
            response_times.append(response_time)
            print(f"   Request {i+1}: {response_time:.2f}ms")
        else:
            print(f"   Request {i+1}: Failed (status {response.status_code})")
            
    except Exception as e:
        print(f"   Request {i+1}: Error - {e}")

if response_times:
    avg_time = sum(response_times) / len(response_times)
    min_time = min(response_times)
    max_time = max(response_times)
    
    print(f"\n📊 Performance Summary:")
    print(f"   Average response time: {avg_time:.2f}ms")
    print(f"   Minimum response time: {min_time:.2f}ms")
    print(f"   Maximum response time: {max_time:.2f}ms")
    print(f"   Successful requests: {len(response_times)}/{num_requests}")

🧪 Testing Performance

Making 10 consecutive requests...
   Request 1: 2044.59ms
   Request 2: 2067.65ms
   Request 3: 2067.21ms
   Request 4: 2073.39ms
   Request 5: 2077.23ms
   Request 6: 2076.92ms
   Request 7: 2082.64ms
   Request 8: 2068.34ms
   Request 9: 2082.70ms
   Request 10: 2053.73ms

📊 Performance Summary:
   Average response time: 2069.44ms
   Minimum response time: 2044.59ms
   Maximum response time: 2082.70ms
   Successful requests: 10/10


## 8. Load Test Data from CSV

Test predictions on actual Titanic test data.

In [67]:
print("🧪 Testing with Real Data\n")

# Load test data from CSV
try:
    test_df = pd.read_csv('../data/raw/test.csv')
    print(f"Loaded {len(test_df)} passengers from test.csv")
    
    # Take first 5 passengers for testing
    sample_passengers = test_df.head(5)
    
    # Convert to the format expected by API
    api_data = []
    for _, row in sample_passengers.iterrows():
        passenger_data = {}
        
        # Map required fields
        passenger_data['Pclass'] = int(row['Pclass'])
        passenger_data['Sex'] = row['Sex']
        
        # Map optional fields (handle NaN values)
        if pd.notna(row['Age']):
            passenger_data['Age'] = float(row['Age'])
        if pd.notna(row['SibSp']):
            passenger_data['SibSp'] = int(row['SibSp'])
        if pd.notna(row['Parch']):
            passenger_data['Parch'] = int(row['Parch'])
        if pd.notna(row['Fare']):
            passenger_data['Fare'] = float(row['Fare'])
        if pd.notna(row['Embarked']):
            passenger_data['Embarked'] = row['Embarked']
        if 'Cabin' in row and pd.notna(row['Cabin']):
            passenger_data['Cabin'] = row['Cabin']
        
        api_data.append(passenger_data)
    
    print(f"\nTesting batch prediction with {len(api_data)} real passengers...")
    
    response = requests.post(
        f"{BASE_URL}/predict/batch",
        headers=headers,
        json={"passengers": api_data}
    )
    
    if response.status_code == 200:
        results = response.json()
        predictions = results['predictions']
        
        print("✅ Real data prediction successful!\n")
        
        # Display results
        for i, (_, passenger) in enumerate(sample_passengers.iterrows()):
            pred = predictions[i]
            status = 'Survived' if pred['prediction'] == 1 else 'Did not survive'
            print(f"Passenger {passenger['PassengerId']}: {status} (prob: {pred['survival_probability']:.3f})")
            print(f"   Class: {passenger['Pclass']}, Sex: {passenger['Sex']}, Age: {passenger.get('Age', 'Unknown')}")
            print()
    else:
        print(f"❌ Real data prediction failed: {response.text}")
        
except FileNotFoundError:
    print("❌ Could not find test.csv file. Make sure it exists in ../data/raw/")
except Exception as e:
    print(f"❌ Error loading real data: {e}")

🧪 Testing with Real Data

Loaded 418 passengers from test.csv

Testing batch prediction with 5 real passengers...
❌ Real data prediction failed: {"detail":[{"type":"missing","loc":["body","passengers",0,"Name"],"msg":"Field required","input":{"Pclass":3,"Sex":"male","Age":34.5,"SibSp":0,"Parch":0,"Fare":7.8292,"Embarked":"Q"}},{"type":"missing","loc":["body","passengers",0,"Ticket"],"msg":"Field required","input":{"Pclass":3,"Sex":"male","Age":34.5,"SibSp":0,"Parch":0,"Fare":7.8292,"Embarked":"Q"}},{"type":"missing","loc":["body","passengers",1,"Name"],"msg":"Field required","input":{"Pclass":3,"Sex":"female","Age":47.0,"SibSp":1,"Parch":0,"Fare":7.0,"Embarked":"S"}},{"type":"missing","loc":["body","passengers",1,"Ticket"],"msg":"Field required","input":{"Pclass":3,"Sex":"female","Age":47.0,"SibSp":1,"Parch":0,"Fare":7.0,"Embarked":"S"}},{"type":"missing","loc":["body","passengers",2,"Name"],"msg":"Field required","input":{"Pclass":2,"Sex":"male","Age":62.0,"SibSp":0,"Parch":0,"Fare":9

## Summary

This notebook has tested:
- ✅ Health check endpoint
- ✅ Model information endpoint
- ✅ Single passenger predictions
- ✅ Batch passenger predictions
- ✅ Error handling and validation
- ✅ API authentication
- ✅ Performance testing
- ✅ Real data predictions

The FastAPI server should now be fully validated and ready for production use!